In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     

In [3]:
import pandas as pd
import numpy as np

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# SUMMARIZATION

In [ ]:
"""from huggingface_hub import list_datasets
ds_lists=list_datasets()
num = 0
for ds in ds_lists:
  print(ds)
  if (num>10):
    break
  num+=1

In [13]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0" )
print(dataset)
print(f"Features: {dataset['train'].column_names}")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})
Features: ['article', 'highlights', 'id']


In [14]:
dataset = load_dataset("cnn_dailymail", "3.0.0" , split = "train[:20%]")
#dataset[20]

In [ ]:
#dataset.shape

(8613, 3)

In [ ]:
#dataset[20]

In [15]:
def preprocess_txt(text_data):
  for txt in ['article','highlights']:
    text_data[txt] = text_data [txt].lower().replace("\\","").replace("/","").replace("\n","").replace("``","")
  return(text_data)

In [16]:
dataset =  dataset.train_test_split(test_size=0.1, seed =20)

In [17]:
ds = dataset.map(preprocess_txt)
ds

Map:   0%|          | 0/51680 [00:00<?, ? examples/s]

Map:   0%|          | 0/5743 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 51680
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 5743
    })
})

In [18]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
article = ds["train"]["article"][0]
print( f"article is  : {article}")

highlights=ds["train"]["highlights"][0]
print( f"highlights is :  {highlights}")

article is  : anaheim, california (cnn) -- disney's cars land opens june 15 at disney california adventure, and it's big, bold and amazing. as mater says, "if i'm lying, i'm crying." kids of all ages will rev their engines and step on the gas. cars land is the capstone of a five year, estimated $1.1 billion dollar reimagining of disney california adventure. while cars land shifts california adventure into the fast lane, only time will tell if it will save the town (as lightning mcqueen did for radiator springs) and turn disneyland into the world-class destination it aspires to be. no doubt the top dogs at disney hope the renewed focus on movie magic will mean magic for the resort that executives have admitted is not up to brand standards. cars land is the centerpiece to the relaunch of california adventure. during the past five years, disney has added more than 20 attractions, including little mermaid and toy story rides, upgraded hotels and new shops. the upgrades are important consid

In [5]:
full_summaries = {}
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, set_seed
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
models = ["t5-small","t5-large","facebook/bart-large-cnn","google/pegasus-cnn_dailymail"]
for model in models:
  summarizer = pipeline ("summarization", model = model, truncation = True)
  summary_txt = summarizer ( "summarize:"+article)
  full_summaries[model] = "\n".join(
    sent_tokenize(summary_txt[0]["summary_text"].replace(" .<n>", ".\n")))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
full_summaries

{'t5-large': "disney's cars land opens june 15 at disney california adventure .\nit's the centerpiece of a five year, estimated $1.1 billion dollar reimagining of the park .\ncars land embraces route 66, radiator falls and the cadillac mountains .",
 'model': 'Cars land is the capstone of a five year, estimated $1.1 billion dollar reimagining .<n> radiator springs racers is a giant slot car system that also features elements that take place in the dark .<n>in this update of the flying saucers concept, giant tires float on a cushion of air supplied by a bunch of small air jets .',
 't5-small': "cars land is the capstone of a five year reimagining of disney california adventure .\nthe resort has added more than 20 attractions, including little mermaid and toy story rides .\nnostalgia doesn't appeal to the younger audience, says cnn's mater .",
 'facebook/bart-large-cnn': "Cars land is the centerpiece to the relaunch of california adventure.\nradiator springs racers is a giant slot car sy

In [21]:
!pip install evaluate
!pip install rouge_score
import evaluate
rouge = evaluate.load("rouge")
rouge

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9da27290dfc5a1dad657d4d2c8ba527e846b7a3c0401e25bd9940f61d67fb404
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
articles = ds["test"]["article"]
articles_summary = ds["test"]["highlights"]

from tqdm import tqdm
summaries = []
for i, text in enumerate (tqdm (articles[:20])):
  summary_txt = summarizer ( "summarize:"+text)
  summaries.append(summary_txt[0]["summary_text"])

100%|██████████| 20/20 [14:43<00:00, 44.17s/it]


In [ ]:
rouge.compute(predictions = summaries, references= articles_summary[:20], use_stemmer=True, use_aggregator=True)

{'rouge1': 0.3531031467194583,
 'rouge2': 0.15786956470877753,
 'rougeL': 0.2513701424669881,
 'rougeLsum': 0.24888418033907472}

In [ ]:
import pandas as pd
df=pd.DataFrame(list(zip (summaries , articles_summary[:20])), columns=["precitions", "actual"])
df.head(20)

,precitions,actual
0,"Tornado hits pensacola, Florida, destroying ho...","at least four people hurt in pensacola, florid..."
1,John hickenlooper was named one of the country...,john hickenlooper says there is a repulsion fo...
2,"""i understood what i was doing wrong but i did...",new: wikileaks founder calls manning's apology...
3,"""hitchcock"" is an oscar wannabe scripted by jo...",anthony hopkins stars as the famous director a...
4,kengo kuma is one of Japan's leading architect...,japanese architect has popularized sensitive j...
5,afghan presidential candidate abdullah abdulla...,abdullah abdullah is in the lead with 82% of t...
6,petroleum giant bp comes under congressional s...,"obama received $71,000 in 2008 election cycle,..."
7,Ryou-un maru caught fire and capsized in more ...,"the ship sunk in more than 6,000 feet of water..."
8,parts of the state have been drenched with up ...,new: bodies recovered from a flooded house an...
9,oberstar was elected in 1974 as a member of th...,jim oberstar represented minnesota's 8th congr...


#FineTune T5

In [ ]:
model="t5-small"
from  transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained (model)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM,  Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model)

NameError: name 'model' is not defined

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
def pre_fn(fine_data):
  inputs=["summarize:" + doc for doc in fine_data["article"]]
  model_inputs=tokenizer(inputs, max_length=1024, truncation = True)
  labels= tokenizer(text_target= fine_data["highlights"],max_length=128, truncation = True)
  model_inputs['labels']=labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_ds =  ds.map(pre_fn, batched = True)

Map:   0%|          | 0/51680 [00:00<?, ? examples/s]

Map:   0%|          | 0/5743 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 51680
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5743
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator= DataCollatorForSeq2Seq(tokenizer = tokenizer , model = model)

In [ ]:
def rouge_metrics (eval_pred):
  pred, actual_labels = eval_pred
  decoded_pred = tokenizer.batch_decode (pred, skip_special_tokens = True)

  actual_labels = np.where(actual_labels !=-100, actual_labels , tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode (actual_labels, skip_special_tokens = True)

  result = rouge.compute(predictions = decoded_pred, references = decoded_labels , use_stemmer = True)
  pred_length = [np.count_nonzero(pred != tokenizer.pad_token_id ) for pred in predictions]
  return {k: round(v,2) for k,v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "new_summary_model",
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub = True,
)

In [ ]:
trainer = Seq2SeqTrainer( model = model ,
                         args = training_args,
                          train_dataset = tokenized_ds["train"],
                          eval_dataset = tokenized_ds ["test"],
                          tokenizer = tokenizer,
                          data_collator = data_collator,
                          compute_metrics =rouge_metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


NameError: name 'rouge' is not defined

In [ ]:
trainer.push_to_hub()

In [6]:
summarizer = pipeline("summarization",model = 'karthiksab/new_summary_model' , truncation = True)
#txt = ("summarize: "+ article)
#summary = summarizer(txt)
#summary

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

NameError: name 'article' is not defined

In [9]:
from transformers import AutoTokenizer
hugging_path="karthiksab/new_summary_model"
tokenizer = AutoTokenizer. from_pretrained(hugging_path)
#input_ids = tokenizer("summarize"+example_text, return_tensors="pt", truncation=True).input_ids


In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(hugging_path)
#outputs = model.generate(input_ids, max_new_tokens=128, num_beams=4, early_stopping=True)
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
#result = rouge.compute(predictions = [summary_txt[0]["summary_text"]], references= [ref_txt], use_stemmer=False, use_aggregator=True)
#result

In [19]:
articles = ds["test"]["article"]
articles_summary = ds["test"]["highlights"]

from tqdm import tqdm
summaries = []
for i, text in enumerate (tqdm (articles[:20])):
  summary_txt = summarizer ( "summarize:"+text)
  summaries.append(summary_txt[0]["summary_text"])

100%|██████████| 20/20 [00:33<00:00,  1.66s/it]


In [22]:
rouge.compute(predictions = summaries, references= articles_summary[:20], use_stemmer=True, use_aggregator=True)

{'rouge1': 0.371586046628969,
 'rouge2': 0.17297249226204092,
 'rougeL': 0.274601475902456,
 'rougeLsum': 0.27268010752295424}